# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

```
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json
```

In [8]:
# Import dependencies
from pymongo import MongoClient, errors
from pprint import pprint
import json

In [9]:
# Create an instance of MongoClient
client = MongoClient(port=27017)

In [10]:
# list the databases to confirm 'uk_food' is listed
databases = client.list_database_names()
print("databases:", databases)

databases: ['admin', 'config', 'local', 'uk_food']


In [11]:
# List the collections in the `uk_food` database to ensure `establishments` is there
collections = client.uk_food.list_collection_names()
print("Collections in uk_food database:", collections)

Collections in uk_food database: ['establishments']


In [12]:
# confirm that our new database was created
db = client["uk_food"]

In [15]:
# assign the uk_food database to a variable name
db = client['uk_food']

In [16]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [17]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66a7e32b3be2335ce71d13bb'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [18]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [19]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [20]:
# Insert the new restaurant into the collection
result = db.establishments.insert_one(new_restaurant)

In [21]:
# Check that the new restaurant was inserted
#Restaurant = pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))
print(f'New restaurant inserted with id: {result.inserted_id}')

New restaurant inserted with id: 66a7f0108ebc14b9ada02faa


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [22]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Execute the query
result = db.establishments.find_one(query, projection)

# Print the result
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [23]:
# Update the new restaurant with the correct BusinessTypeID
# Step 1: Find the correct BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
business_type = db.establishments.find_one(query, projection)

if business_type:
    # Step 2: Update the new restaurant's document with the correct BusinessTypeID
    new_restaurant_query = {"BusinessName": "Penang Flavours"}
    update = {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
    
    result = db.establishments.update_one(new_restaurant_query, update)
    
    if result.matched_count > 0:
        print(f'Successfully updated the new restaurant with BusinessTypeID: {business_type["BusinessTypeID"]}')
    else:
        print('No matching document found to update.')
else:
    print('BusinessType "Restaurant/Cafe/Canteen" not found.')

Successfully updated the new restaurant with BusinessTypeID: 1


In [24]:
# Confirm that the new restaurant was updated
confirmation_query = {"BusinessName": "Penang Flavours"}
confirmation_projection = {"BusinessName": 1, "BusinessType": 1, "BusinessTypeID": 1, "_id": 0}
# To find the updated document
updated_restaurant = db.establishments.find_one(confirmation_query, confirmation_projection)

# Print to confirm the update
if updated_restaurant:
    pprint(updated_restaurant)
else:
    print('The new restaurant document was not found.')

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [26]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count_before = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f'Number of establishments in Dover before deletion: {dover_count_before}')

Number of establishments in Dover before deletion: 0


In [27]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f'Number of documents deleted: {delete_result.deleted_count}')


Number of documents deleted: 0


In [28]:
# Check if any remaining documents include Dover
dover_count_after = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f'Number of establishments in Dover after deletion: {dover_count_after}')

Number of establishments in Dover after deletion: 0


In [30]:
# Check that other documents remain with 'find_one'
remaining_document = db.establishments.find_one({"LocalAuthorityName": {"$ne": "Dover"}})

# Print the result to confirm other documents remain
if remaining_document:
    pprint(remaining_document)
else:
    print('No other documents found in the database.')

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66a7e32b3be2335ce71d16a2'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [31]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many(
    {"geocode.longitude": {"$type": "string"}},
    [{"$set": {"geocode.longitude": {"$convert": {"input": "$geocode.longitude", "to": "double", "onError": 0, "onNull": 0}}}}]
)

db.establishments.update_many(
    {"geocode.latitude": {"$type": "string"}},
    [{"$set": {"geocode.latitude": {"$convert": {"input": "$geocode.latitude", "to": "double", "onError": 0, "onNull": 0}}}}]
)

# Step 2: Confirm the update by retrieving one document
updated_document = db.establishments.find_one({"geocode.longitude": {"$type": "double"}})
if updated_document:
    print("Longitude and Latitude data types updated to decimal:")
    pprint(updated_document)
else:
    print("No document with updated data types found.")

Longitude and Latitude data types updated to decimal:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66a7e32b3be2335ce71d16a2'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
   

Use `update_many` to convert `RatingValue` to integer numbers.

In [32]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [33]:
# Change the data type from String to Integer for RatingValue
db.establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [34]:
# Check that the coordinates and rating value are now numbers
check_result = db.establishments.find_one(
    {
        "geocode.longitude": {"$type": "double"},
        "geocode.latitude": {"$type": "double"},
        "RatingValue": {"$type": "int"}
    },
    {"BusinessName": 1, "geocode.longitude": 1, "geocode.latitude": 1, "RatingValue": 1, "_id": 0}
)

# Print the result to confirm
if check_result:
    pprint(check_result)
else:
    print("No document with updated coordinates and rating value found.")

{'BusinessName': 'The Pavilion',
 'RatingValue': 5,
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
